# Trabajo Final - "Titanic"

### Plan de trabajo


1. Apertura archivos y analisis (tamaños, estructura matriz, etc)
    - Analisis estructura de la matriz y cantidad de datos en cada archivo
2. Análisis de datos dentro del train 
    - Cantidad de datos nulos y análisis de datos para determinar conversiones, agregado/borrado/modificación de columnas, identificación de datos "Outliers" (o atípicos) o de necesidad de normalizar por tener magnitudes muy diferentes
    - Definición de acciones a implementar para preparar los datos previo al entrenamiento
3. Borrado de columnas que no utilizaremos

----------1er etapa MATI

4. Completar columnas con datos faltantes
5. Transformar columnas con datos objeto a varias columnas con datos numéricos (ceros y unos)
6. Ejecución y búsqueda del mejor modelo de aprendizaje
7. Ejecución del mejor modelo encontrado
8. Guardado del modelo
9. Apertura  del modelo guardado y prueba con 2 datos nuevos

### 1. Apertura de archivos y análisis

In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_rows', None)

In [3]:
train = pd.read_csv("data/train.csv")
X_test = pd.read_csv("data/test.csv")
y_test = pd.read_csv("data/gender_submission.csv")

#### Analisis estructura de la matriz y datos de los archivos

In [4]:
train.shape

(891, 12)

In [5]:
X_test.shape

(418, 11)

In [6]:
y_test.shape

(418, 2)

>___Conclusiones: Entre train y test hay 1309 pasajeros, train con 891 y test con 418 (60%/40%), con 12 columnas en el train___

>___Dado que el train da 60% y el test 40%, ¿deberiamos unir todo y dividir luego en 80/20 para tener mas datos para entrenar?___

In [7]:
train.head(4)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S


***Columnas y contenidos***

Variable	   Definición		        Llave
- survival	     Supervivencia		0 = No, 1 = Sí
- pclass		Clase de billete	1 = 1º, 2 = 2º, 3 = 3º
- sex		    Sexo	
- age		    Edad en años	
- sibsp		    # de hermanos / cónyuges a bordo del Titanic	
- parch		    # de padres / hijos a bordo del Titanic	
- ticket		Número de billete	
- fare		    Tarifa de pasajero	
- cabin		    Número de cabina	
- embarked	    Puerto de embarque	C = Cherburgo, Q = Queenstown, S = Southampton

In [8]:
train.sort_values(by=["Pclass","Fare"], ascending=[True, True]).groupby("Pclass")["Fare"].mean()

Pclass
1    84.154687
2    20.662183
3    13.675550
Name: Fare, dtype: float64

In [9]:
cortes=pd.cut(train.Pclass,3)

In [10]:
x=train.groupby(["Pclass","Fare"],squeeze=True)["Fare"].value_counts()
#print(x)

<ipython-input-10-d8a004d45806>:1: FutureWarning: The `squeeze` parameter is deprecated and will be removed in a future version.
  x=train.groupby(["Pclass","Fare"],squeeze=True)["Fare"].value_counts()


In [11]:
#train.sort_values(by=["Pclass","Fare"], ascending=[True,True])

In [12]:
#train.Pclass

In [13]:
train.groupby(("Embarked")).count()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin
Embarked,,,,,,,,,,,
C,168,168,168,168,168,130,168,168,168,168,69
Q,77,77,77,77,77,28,77,77,77,77,4
S,644,644,644,644,644,554,644,644,644,644,129


In [14]:
train.groupby(("Embarked")).mean()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
Embarked,,,,,,,
C,445.357143,0.553571,1.886905,30.814769,0.386905,0.363095,59.954144
Q,417.896104,0.389610,2.909091,28.089286,0.428571,0.168831,13.276030
S,449.527950,0.336957,2.350932,29.445397,0.571429,0.413043,27.079812


##### El archivo Test tiene 418 filas por 11 columnas

In [15]:
X_test.shape

(418, 11)

In [16]:
X_test.head(4)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S


In [17]:
y_test.shape

(418, 2)

In [18]:
y_test.head(4)

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0


In [19]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [20]:
train.Pclass.unique()

array([3, 1, 2], dtype=int64)

In [21]:
train.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [22]:
train.Parch.value_counts(dropna=True) 

0    678
1    118
2     80
3      5
5      5
4      4
6      1
Name: Parch, dtype: int64

In [23]:
train.groupby(by=["Parch"]).count()  #(by=["b"]).sum()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Ticket,Fare,Cabin,Embarked
Parch,,,,,,,,,,,
0,678,678,678,678,678,521,678,678,678,141,676
1,118,118,118,118,118,110,118,118,118,39,118
2,80,80,80,80,80,68,80,80,80,23,80
3,5,5,5,5,5,5,5,5,5,0,5
4,4,4,4,4,4,4,4,4,4,1,4
5,5,5,5,5,5,5,5,5,5,0,5
6,1,1,1,1,1,1,1,1,1,0,1


In [24]:
np.count_nonzero(train.Parch)

213

In [25]:
np.count_nonzero(train.SibSp) 

283

In [26]:
train["Flia"]=train["SibSp"]+train["Parch"]
X_test["Flia"]=X_test["SibSp"]+X_test["Parch"]

In [27]:
X_test.head(1)
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
 11  Flia         418 non-null    int64  
dtypes: float64(2), int64(5), object(5)
memory usage: 39.3+ KB


In [28]:
train.head(4)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Flia
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1


In [29]:
np.sum(train.Flia)

806

In [30]:
np.sum(train.Parch)

340

In [31]:
np.sum(train.SibSp)

466

In [32]:
train.groupby(["Flia"]).sum()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
Flia,,,,,,,
0,245946,163,1289,13017.00,0,0,11407.3238
1,66467,89,309,4363.42,123,38,8032.9547
2,45732,59,227,2421.33,89,115,4048.6332
3,14916,21,60,493.42,39,48,1591.0708
4,5518,3,40,229.00,32,28,871.4168
5,8803,3,57,405.00,62,48,1621.9000
6,4269,4,36,182.00,39,33,352.4000
7,2363,0,18,94.00,26,16,281.4000
10,3372,0,21,0.00,56,14,486.8500


##### Conclusiones
- Sacar columna Name, Ticket,Fare, Cabin, Sibsp y Parch (porque estos últimos dos estan ya sumados en Flia

In [33]:
# Borrado de columnas que no utilizaremos. 
train = train.drop(['Name', 'Ticket', 'Fare', 'Cabin','SibSp','Parch'], axis=1) 
train.head(20)

,PassengerId,Survived,Pclass,Sex,Age,Embarked,Flia
0,1,0,3,male,22.0,S,1
1,2,1,1,female,38.0,C,1
2,3,1,3,female,26.0,S,0
3,4,1,1,female,35.0,S,1
4,5,0,3,male,35.0,S,0
5,6,0,3,male,NaN,Q,0
6,7,0,1,male,54.0,S,0
7,8,0,3,male,2.0,S,4
8,9,1,3,female,27.0,S,2
9,10,1,2,female,14.0,C,1


Split del train en X_train e y_train

In [34]:
y_train=train[["Survived"]]
y_train.head(4)

,Survived
0,0
1,1
2,1
3,1


In [35]:
X_train=train.drop(["Survived"],axis=1)
X_train.head(4)

,PassengerId,Pclass,Sex,Age,Embarked,Flia
0,1,3,male,22.0,S,1
1,2,1,female,38.0,C,1
2,3,3,female,26.0,S,0
3,4,1,female,35.0,S,1


- Sex y Embarked: transformar a 2/3 columnas numéricas con unos y ceros
- Age: Agregar 177 datos faltantes en train y los faltantes en test   

In [36]:
X_train.head(1)

,PassengerId,Pclass,Sex,Age,Embarked,Flia
0,1,3,male,22.0,S,1


In [37]:
X_test.head(1)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Flia
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,0


In [38]:
# CORRECCIÓN AL X_test.
# Borrado de columnas que no utilizaremos. 
X_test = X_test.drop(['Name', 'Ticket', 'Fare', 'Cabin','SibSp','Parch'], axis=1) 
X_test.head(1)

,PassengerId,Pclass,Sex,Age,Embarked,Flia
0,892,3,male,34.5,Q,0


In [39]:
# Con onehotencoder
# Transformar columnas con datos objeto a varias columnas con datos numéricos (ceros y unos).
# Esas serían las columnas: sex, embarked.
# Recordar ahcerlo primero en train y luego en test
from sklearn.compose import ColumnTransformer 
from sklearn.preprocessing import OneHotEncoder
# Definimos el transformador para OneHotEncoder de foma muy similar a como trabajamos SimpleImputer o kNNImputer
tdummy_2 = ("onehot",OneHotEncoder(sparse = False), ['Sex', 'Embarked'])

# Creamos el transformador de columnas
transformador_columnas_dummy2 = ColumnTransformer(transformers = [tdummy_2], remainder = 'passthrough')
# Lo entrenamos con fit en X_train
transformador_columnas_dummy2.fit(X_train)
# Lo aplicamos al X_train
dummy_2 = transformador_columnas_dummy2.transform(X_train)
dummy_2


array([[ 0.,  1.,  0., ...,  3., 22.,  1.],
       [ 1.,  0.,  1., ...,  1., 38.,  1.],
       [ 1.,  0.,  0., ...,  3., 26.,  0.],
       ...,
       [ 1.,  0.,  0., ...,  3., nan,  3.],
       [ 0.,  1.,  1., ...,  1., 26.,  0.],
       [ 0.,  1.,  0., ...,  3., 32.,  0.]])

In [40]:
# Lo aplicamos al y_test
dummy_2 = transformador_columnas_dummy2.transform(X_test)
dummy_2


array([[ 0. ,  1. ,  0. , ...,  3. , 34.5,  0. ],
       [ 1. ,  0. ,  0. , ...,  3. , 47. ,  1. ],
       [ 0. ,  1. ,  0. , ...,  2. , 62. ,  0. ],
       ...,
       [ 0. ,  1. ,  0. , ...,  3. , 38.5,  0. ],
       [ 0. ,  1. ,  0. , ...,  3. ,  nan,  0. ],
       [ 0. ,  1. ,  1. , ...,  3. ,  nan,  2. ]])

In [41]:
# Completar columna Age con modelo KnnImputer con datos faltantes. 
# Recordar ahcerlo primero en train y luego en test



#### Búsqueda del mejor modelo

In [42]:
# Ejecución del mejor modelo encontrado.


In [43]:
# Guardado del modelo.


In [44]:
# Apertura del modelo guardado y prueba con 2 datos nuevos.
